In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path
import os
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#tokenization for wordcloud, ect..
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tylergehbauer/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
netflix_df = pd.read_csv("../data/netflix.csv")

In [6]:
sentiments_netflix = []#creates empty list

#Going to use VADER’s SentimentIntensityAnalyzer() to see how many 'Neutral (neu)' , 'Positive (pos)', and 'Negatvie(neg)'
# words there are for each article. 
for articles in netflix_df['critics_consensus']: #goes through each artilce in df and applies polarity
    try:
        text = articles #accessing each articles 'content' which contains the article itself in 'bitcoin_news'
        results = analyzer.polarity_scores(text) #this is how get compound, pos, neg, and nue polarity scores
        #during imports we set analyzer equal to SentimentIntensityAnalyzer() 
        #'results' uses this function to find polarity score for each article in bitcoin_news
        compound = results["compound"] #gets compound polarity score
        pos = results["pos"] #gets postive polarity score
        neu = results["neu"] #gets neutral polarity score
        neg = results["neg"] #gets negative polarity score

        sentiments_netflix.append({
            "text": text,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        }) #appends a dictionary inside the list-(sentiments) so a dataframe can be created
        #this appends all polarity score we got above
    except AttributeError:
        pass
#rinse and repeat for each article    
netflix = pd.DataFrame(sentiments_netflix)#creates a dataframe using the list of dicts
netflix = netflix.join(netflix_df['movie_title'])
netflix = netflix.join(netflix_df['original_release_date'])
netflix = netflix.join(netflix_df['streaming_release_date'])
netflix = netflix.drop(columns = ['Positive', 'Negative', 'Neutral'])

In [7]:
netflix[["original_release_date", "streaming_release_date"]] = netflix[["original_release_date", "streaming_release_date"]].apply(pd.to_datetime)


In [8]:
netflix.head()

,text,Compound,Positive,Negative,Neutral,movie_title,original_release_date,streaming_release_date
0,Thanks to director Zak Hilditch's patient stor...,0.8588,0.322,0.000,0.678,18 Presents (18 Regali),NaT,2020-05-19
1,22 July offers a hard-hitting close-up look at...,-0.6124,0.057,0.163,0.780,1922,2017-10-20,2017-10-20
2,37 Seconds is a rare and refreshing film that ...,0.7351,0.323,0.000,0.677,22-Jul,2018-10-10,2018-10-10
3,"Delicately sketched but thematically rich, 6 B...",0.8779,0.347,0.000,0.653,37 Seconds,NaT,2020-01-31
4,"6 Underground is loud, frenetic, and finally p...",-0.5423,0.000,0.123,0.877,6 Balloons,NaT,2018-04-06
